<a href="https://colab.research.google.com/github/compartia/AI-tecture/blob/master/miro_openai_pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Miro and Chat GPT

 status: **DRAFT**

 **by:** Artem Zaborskiy


 system that integrates AI-generated content with Miro (https://miro.com/), a collaborative online whiteboard platform. It specifically handles the part of the workflow where an AI-generated response is posted back to the Miro board, enhancing the interactive and dynamic capabilities of the board by incorporating AI insights directly into the collaborative workspace.



### TODOs:

1. sort nodes to process, proritize, what propmps to process first
1. cache processing results,
1. ~~check dates or checksums~~
1. check for self-connected nodes
1. check for prompt-prompt connectors
1. do not reload the entire board after widget created: add new widget and its connector to dataframes
1. ~~create output shape if propmpt node has no output connectors~~

## Prepare OPEN AI client

In [ ]:
from IPython.core.display import display, HTML

In [ ]:
!pip install --upgrade openai typing-extensions

In [ ]:
import openai
from google.colab import userdata

ai_client = openai.OpenAI(api_key=userdata.get('OP_API')  )

In [ ]:
BOARD_ID = 'uXjVN61x8Pg='
DEBUG_OPENAI = False

### Test open AI api calls

In [ ]:
%%time

if DEBUG_OPENAI:
  completion = ai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair. Your answer is in a form of list of <p> html tags."},
      {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ]
  )

  # print(completion.choices[0].message)
  display(HTML(completion.choices[0].message.content))

In [ ]:
%%time

if DEBUG_OPENAI:
  completion = ai_client.chat.completions.create(
    model="gpt-4",
    messages=[
      {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair. Your answer is in a form of list of <p> html tags."},
      {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ]
  )
  #
  # print(completion.choices[0].message)
  display(HTML(completion.choices[0].message.content))

In [ ]:
%%time

if DEBUG_OPENAI:
  completion = ai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
      {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair. Your answer is in a form of list of <p> html tags."},
      {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ]
  )

  display(HTML(completion.choices[0].message.content))

## OpenAI API tools

In [ ]:
def ask_ai(prompt, argument_context):

  completion = ai_client.chat.completions.create(
    model="gpt-4",
    messages=[
      {"role": "system", "content": f"{prompt}. Your answer is in a form of list of <p> HTML tags and has NO newline '\n' symbols."},
      {"role": "user", "content": argument_context}
    ]
  )
  print(completion.choices)
  return completion.choices[0].message.content

if DEBUG_OPENAI or True:
  r = ask_ai('you are a female philosopher. Answer in 2 words.', 'what is the meaning of life?')
  print(r)
  display(HTML(r))
  # display(HTML(r))

In [ ]:
if DEBUG_OPENAI or True:
  r = ask_ai('you are a male philosopher. Answer in 2 words.', 'what is the meaning of life?')
  print(r)
  display(HTML(r))
  # display(HTML(r))

# Miro client
https://developers.miro.com/reference/api-reference

In [ ]:
import requests
import pandas as pd


class MiroClient:
  def __init__(self, token, board_id):
    self.board_id = board_id
    self.token = token
    self.base_url = f"https://api.miro.com/v2/boards/{board_id}"

    self.headers = {
        "accept": "application/json",
        "authorization": f"Bearer {self.token}"
    }


  def get_items(self, cursor=None, limit=50):
    limit_p = f'limit={limit}'

    url = f"{self.base_url}/items?{limit_p}"
    if cursor is not None:
      url = f"{self.base_url}/items?cursor={cursor}&{limit_p}"

    response = requests.get(url, headers=self.headers)
    return response

  def get_connectors(self, cursor=None, limit=50):
    limit_p = f'limit={limit}'

    url = f"{self.base_url}/connectors?{limit_p}"
    if cursor is not None:
      url = f"{self.base_url}/connectors?cursor={cursor}&{limit_p}"

    response = requests.get(url, headers=self.headers)
    return response


  def get_all_pages(self, page_method):

    cursor = None

    max_steps = 30
    step = 0
    while True:
      step += 1
      response = page_method(cursor)
      rj = response.json()
      yield rj
      cursor = rj.get('cursor')
      if cursor is None or step > max_steps:
        break


miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)


## Getting all connectors

In [ ]:
def decode_connectors(page:dict):
  # print(page)
  for i in page['data']:
    yield i['id'], i.get('startItem', {}).get('id'), i.get('endItem', {}).get('id')


##-----
## test it
one_page_con = miro_client.get_connectors(limit=10).json()
x = [i for i in decode_connectors(one_page_con)]
x

In [ ]:
def get_all_connectors(self):
  for page in self.get_all_pages(self.get_connectors):
    for i in decode_connectors(page):
      yield i


def all_connectors_as_df(miro_client):
  all_connectors_iter = miro_client.get_all_connectors()
  miro_client.connectors_df = pd.DataFrame(all_connectors_iter, columns=['id','id_from', 'id_to'])
  miro_client.connectors_df = miro_client.connectors_df.set_index('id')


  return miro_client.connectors_df


MiroClient.get_all_connectors = get_all_connectors
MiroClient.all_connectors_as_df = all_connectors_as_df



##-----
miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)

connectors_df = miro_client.all_connectors_as_df()
connectors_df


## Getting all shapes (widgets)

In [ ]:
def decode_items(page:dict):

  for i in page['data']:
    # print(type(i))
    if type(i) == dict :
      yield i['id'], i['type'], i['data'].get('shape', '_undefined_'), i['data'].get('content'), i['modifiedAt'], i['geometry'], i['position']

    else:
      print('NOT A DICT!!', type(i), i)
      decode_items(i)



def get_all_items(self):
  for page in self.get_all_pages(self.get_items):
    # print( page.get('cursor'), page.get('size'), page.get('total') )
    for i in decode_items(page):
      yield i
      # print(i)


MiroClient.get_all_items = get_all_items

In [ ]:

def all_items_as_df(miro_client):
  items_iter = miro_client.get_all_items()

  df = pd.DataFrame(items_iter, columns=['id','type', 'shape', "contents", 'modifiedAt', 'geometry', 'position'])
  df = df.set_index('id')
  df['modifiedAt_date'] = pd.to_datetime(df['modifiedAt'])

  miro_client.shapes_df = df
  return miro_client.shapes_df


MiroClient.all_items_as_df = all_items_as_df
# --------

miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)
shapes_df = miro_client.all_items_as_df()

In [ ]:
shapes_df

### Update board item

In [ ]:
def get_shape_info(self, shape_id):
  url = f"{self.base_url}/shapes/{shape_id}"
  response = requests.get(url, headers=self.headers)
  print(response.text)

def get_stiky_note_info(self, shape_id):
  url = f"{self.base_url}/sticky_notes/{shape_id}"
  response = requests.get(url, headers=self.headers)
  print(response.text)


def update_item(self, shape_id, item_type, content, shape=None, color = None):

  url = None

  payload = {
      "data": {
          "content": content
      }
  }


  if color is not None:
    payload['style']={}
    payload['style']['fillColor'] = color

    if item_type!='sticky_note':
      payload['style']['fillOpacity'] = "1.0"


  if item_type=='sticky_note':
    url = f"{self.base_url}/sticky_notes/{shape_id}"
  elif item_type=='shape':
    url = f"{self.base_url}/shapes/{shape_id}"
    if shape is not None:
      payload['data']['shape'] = shape

  print(payload)


  response = requests.patch(url, json=payload, headers=self.headers)
  print(response.text)
  return response



MiroClient.update_item = update_item
MiroClient.get_shape_info = get_shape_info
MiroClient.get_stiky_note_info = get_stiky_note_info

#-----

miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)

# if False:

  # miro_client.get_stiky_note_info('3458764578213963472')
# test_id = '3458764578216605194'
# miro_client.update_item( test_id, item_type=shapes_df.at[test_id, 'type'], content=completion.choices[0].message.content,  color='#ff0000')

In [ ]:
def create_connector(self, form, to, text):
  import requests

  url = f"{self.base_url}/connectors"

  payload = {
      "startItem": {
          "id": form,
          "snapTo": "right"
      },
      "endItem": {
          "id": to,
          "snapTo": "auto"
      },
      "style": { "endStrokeCap": "arrow" },
      "shape": "curved",
      "captions": [{ "content": text }]
  }

  response = requests.post(url, json=payload, headers=self.headers)
  return response


def create_stiky_note(self, contents, pos):

  url = f"{self.base_url}/sticky_notes"

  payload = {
    "data": {
        "content": contents,
        "shape": "rectangle"
    },
    "style": {
        "fillColor": "pink",
        "textAlign": "left"
    },
    "position": {
        "x": pos[0],
        "y": pos[1]
    },
    "geometry": {
        "height": pos[2]
        # "width": pos[3]
    }
  }

  response = requests.post(url, json=payload, headers=self.headers)
  return response


MiroClient.create_stiky_note = create_stiky_note
MiroClient.create_connector = create_connector


# --------
miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)
# response = miro_client.create_stiky_note( "TESTCREATE", (20,20,400,400) )
# response.json()['id']

# Processing the board


In [ ]:
miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)

### Get shapes with prompts

In [ ]:
prompts_df = shapes_df[shapes_df['shape']=='parallelogram']
prompts_df

### Find connectors of prompts-shapes (connectors of prallelograms)

Updating Miro board widgets with an answers generated by an AI model. Here's a breakdown of its functionality:

1. **Identify Output Links**: Finding output links of the prompt widget from the `connectors_df` DataFrame where the `id_from` matches the `prompt_id`. If no outgoing connectors are found it creates a widget on the board and links it to the propt widsget

2. **Update Widget**:  the target widget(s) is updated with AI-generated answer. This update is performed by calling the `update_item` method of the `miro_client` object



In [ ]:
def build_output_shape(shape_id, contents = 'Waiting for AI to answer'):

  global shapes_df

  shape = shapes_df.loc [ shape_id ]

  pos = shape['position']
  geo = shape['geometry']

  size = max(geo['width'], geo['height'])

  response = miro_client.create_stiky_note( contents, (pos['x'] +  geo['width'] + size, pos['y'], size, size) )
  a_id = response.json()['id']

  response = miro_client.create_connector( shape_id,  a_id, 'AI answer' )

  #TODO: OPTIMIZE!!
  shapes_df = miro_client.all_items_as_df()

  return a_id

# test------
# build_output_shape('3458764578079690837', "TESTCREATE")






In [ ]:
def get_output_shape_id(prompt_id):
  out_links = connectors_df[ connectors_df['id_from']==prompt_id]
  if len(out_links) > 0:
    _items = shapes_df.loc [ out_links['id_to'] ]
    return _items.iloc[0].name

In [ ]:
def get_incoming_text(incoming_links):

  incoming_items = shapes_df.loc [incoming_links['id_from'] ]
  incoming_items = incoming_items.sort_values(by='position', key=lambda x: x.map(lambda d: d['y']), ascending=True)

  concatenated_text = '\n\n'.join(incoming_items.contents)
  # print('ii---- ',len(incoming_items), concatenated_text)

  return concatenated_text

In [ ]:
def get_incoming_max_date(incoming_links):
  incoming_items = shapes_df.loc [incoming_links['id_from'] ]
  return incoming_items.modifiedAt_date.max()

# 3458764578299251402

In [ ]:
_incoming_links = connectors_df[ connectors_df['id_to']=='3458764578299251402']

incoming_items = shapes_df.loc [ _incoming_links['id_from'] ]

# import datetime
print(incoming_items.modifiedAt_date.max())
# datetime.datetime.fromisoformat(incoming_items.modifiedAt.max())

In [ ]:


def get_color_for_state(out_shape_id, state):
  item_type = shapes_df.at[out_shape_id, 'type']

  if state==0:
    if item_type=='sticky_note':
      color = 'red'
    else:
      color = '#ff5566'

  if state==1:
    if item_type=='sticky_note':
      color = 'light_green'
    else:
      color = '#ccffdd'

  return color


def build_answer(prompt_id):
  default_contents='waiting for AI to answer ....'
  _propmt_text = shapes_df.loc[prompt_id]['contents']

  incoming_links = connectors_df[ connectors_df['id_to']==prompt_id]
  incoming_max_date = get_incoming_max_date(incoming_links)

  # if there are more than 1 input, merge them
  in_text = get_incoming_text(incoming_links)

  update_is_required = False

  out_shape_id = get_output_shape_id(prompt_id)
  if out_shape_id is None:
    out_shape_id = build_output_shape(prompt_id, contents =default_contents)
    update_is_required = True

  out_date = shapes_df.at[out_shape_id, 'modifiedAt_date']
  prompt_date = shapes_df.at[prompt_id, 'modifiedAt_date']

  update_is_required = (out_date < incoming_max_date) or (prompt_date > out_date) or update_is_required

  if update_is_required:
    item_type = shapes_df.at[out_shape_id, 'type']
    color = get_color_for_state(out_shape_id, 0)
    res = miro_client.update_item( out_shape_id, item_type=item_type, content=default_contents, color = color)

    answer_text = ask_ai(_propmt_text, in_text)

    try:
      display(HTML(r))
    except:
      print('cannot render html', r)

    color = get_color_for_state(out_shape_id, 1)
    res = miro_client.update_item( out_shape_id, item_type=item_type, content=answer_text,  shape='round_rectangle', color = color)
  else:
    print('ignoring', prompt_id, f'because update date : {incoming_max_date} < {out_date}')

  return r


#TODO: sort propmpts first
for prompt_id in prompts_df.index:
  print('-'*40)
  print(prompt_id, 'processing')
  answer_text = build_answer(prompt_id)

